# SIMD

SIMD stands for **"Single Instruction Multiple Data"** and falls into the category of instruction level parallelism (vector instructions). Consider this simple example where `A`, `B`, and `C` are vectors:

In [9]:
function vector_add(A, B, C)
    for i in eachindex(A, B, C)
        @inbounds A[i] = B[i] + C[i]
    end
end

vector_add (generic function with 1 method)


The idea behind SIMD is to perform the add instruction on multiple elements at the same time (instead of separately performing them one after another). The process of splitting up the simple loop addition into multiple vector additions is often denoted as "loop vectorization". Since each vectorized addition happens at instruction level, i.e. within a CPU core, the feature set of the CPU determines how many elements we can process in one go.

<img src="../../imgs/simd_vaddpd.png" width=300px>
<img src="../../imgs/simd_register_width.png" width=400px>

(**Source:** Node-level performance engineering course by [NHR@FAU](https://hpc.fau.de/))

Let's check which "advanced vector extensions" (AVX) the system supports.

In [1]:
using CpuId
cpuinfo()

| Cpu Property       | Value                                                      |
|:------------------ |:---------------------------------------------------------- |
| Brand              | Intel(R) Core(TM) i7-9750H CPU @ 2.60GHz                   |
| Vendor             | :Intel                                                     |
| Architecture       | :Kabylake                                                  |
| Model              | Family: 0x06, Model: 0x9e, Stepping: 0x0a, Type: 0x00      |
| Cores              | 6 physical cores, 12 logical cores (on executing CPU)      |
|                    | Hyperthreading hardware capability detected                |
| Clock Frequencies  | 2600 / 4500 MHz (base/max), 100 MHz bus                    |
| Data Cache         | Level 1:3 : (32, 256, 12288) kbytes                        |
|                    | 64 byte cache line size                                    |
| Address Size       | 48 bits virtual, 39 bits physical                          |
| SIMD               | 256 bit = 32 byte max. SIMD vector size                    |
| Time Stamp Counter | TSC is accessible via `rdtsc`                              |
|                    | TSC runs at constant rate (invariant from clock frequency) |
| Perf. Monitoring   | Performance Monitoring Counters (PMC) revision 4           |
|                    | Available hardware counters per logical core:              |
|                    | 3 fixed-function counters of 48 bit width                  |
|                    | 4 general-purpose counters of 48 bit width                 |
| Hypervisor         | No                                                         |


In [2]:
filter(x -> contains(string(x), "AVX"), cpufeatures())

2-element Vector{Symbol}:
 :AVX
 :AVX2

In [6]:
SIZE = 512 * 100

51200

In [7]:
A = rand(Float64, SIZE)
B = rand(Float64, SIZE)
C = rand(Float64, SIZE);

In [10]:
@code_native debuginfo=:none syntax=:intel vector_add(A,B,C)

	.section	__TEXT,__text,regular,pure_instructions
	.build_version macos, 12, 0
	.globl	_japi1_vector_add_2338          ## -- Begin function japi1_vector_add_2338
	.p2align	4, 0x90
_japi1_vector_add_2338:                 ## @japi1_vector_add_2338
	.cfi_startproc
## %bb.0:                               ## %top
	push	rbp
	.cfi_def_cfa_offset 16
	.cfi_offset rbp, -16
	mov	rbp, rsp
	.cfi_def_cfa_register rbp
	push	r15
	push	r14
	push	r13
	push	r12
	push	rbx
	and	rsp, -32
	sub	rsp, 128
	.cfi_offset rbx, -56
	.cfi_offset r12, -48
	.cfi_offset r13, -40
	.cfi_offset r14, -32
	.cfi_offset r15, -24
	mov	rbx, rsi
	vxorpd	xmm0, xmm0, xmm0
	vmovapd	ymmword ptr [rsp], ymm0
	mov	qword ptr [rsp + 32], 0
	mov	qword ptr [rsp + 96], rsi
	movabs	rax, 140703379165141
	mov	edi, 260
	vzeroupper
	call	rax
	mov	r14, rax
	mov	qword ptr [rsp], 12
	mov	rax, qword ptr [rax]
	mov	qword ptr [rsp + 8], rax
	mov	rax, rsp
	mov	qword ptr [r14], rax
	mov	rdx, qword ptr [rbx]
	mov	rax, qword ptr [rbx + 8]
	mov	rcx, qword p

## It's not always so simple: Reduction

In [11]:
function vector_dot(B, C)
    a = zero(eltype(B))
    for i in eachindex(B,C)
        @inbounds a += B[i] * C[i]
    end
    return a
end

vector_dot (generic function with 1 method)

In [12]:
@code_native debuginfo=:none syntax=:intel vector_dot(B, C)

	.section	__TEXT,__text,regular,pure_instructions
	.build_version macos, 12, 0
	.globl	_julia_vector_dot_2396          ## -- Begin function julia_vector_dot_2396
	.p2align	4, 0x90
_julia_vector_dot_2396:                 ## @julia_vector_dot_2396
	.cfi_startproc
## %bb.0:                               ## %top
	push	rbp
	.cfi_def_cfa_offset 16
	.cfi_offset rbp, -16
	mov	rbp, rsp
	.cfi_def_cfa_register rbp
	push	r15
	push	r14
	push	r13
	push	r12
	push	rbx
	and	rsp, -32
	sub	rsp, 96
	.cfi_offset rbx, -56
	.cfi_offset r12, -48
	.cfi_offset r13, -40
	.cfi_offset r14, -32
	.cfi_offset r15, -24
	mov	r15, rsi
	mov	r12, rdi
	vxorpd	xmm0, xmm0, xmm0
	vmovapd	ymmword ptr [rsp], ymm0
	movabs	rax, 140703379165141
	mov	edi, 260
	vzeroupper
	call	rax
	mov	r14, rax
	mov	qword ptr [rsp], 8
	mov	rax, qword ptr [rax]
	mov	qword ptr [rsp + 8], rax
	mov	rax, rsp
	mov	qword ptr [r14], rax
	mov	rbx, qword ptr [r12 + 8]
	mov	r13, qword ptr [r15 + 8]
	cmp	rbx, r13
	jne	LBB0_6
## %bb.1:                          

Note the `vaddsd` instruction and usage of `xmmi` registers (128 bit).

How could this loop be vectorized?

In [13]:
function vector_dot_unrolled4(B, C)
    a1 = zero(eltype(B))
    a2 = zero(eltype(B))
    a3 = zero(eltype(B))
    a4 = zero(eltype(B))
    @inbounds for i in 1:4:length(B)-4
        a1 += B[i] * C[i]
        a2 += B[i+1] * C[i+1]
        a3 += B[i+2] * C[i+2]
        a4 += B[i+3] * C[i+3]
    end
    return a1+a2+a3+a4
end

vector_dot_unrolled4 (generic function with 1 method)

In [14]:
@code_native debuginfo=:none syntax=:intel vector_dot_unrolled4(B, C)

	.section	__TEXT,__text,regular,pure_instructions
	.build_version macos, 12, 0
	.globl	_julia_vector_dot_unrolled4_2422 ## -- Begin function julia_vector_dot_unrolled4_2422
	.p2align	4, 0x90
_julia_vector_dot_unrolled4_2422:       ## @julia_vector_dot_unrolled4_2422
	.cfi_startproc
## %bb.0:                               ## %top
	push	r14
	.cfi_def_cfa_offset 16
	push	rbx
	.cfi_def_cfa_offset 24
	sub	rsp, 8
	.cfi_def_cfa_offset 32
	.cfi_offset rbx, -24
	.cfi_offset r14, -16
	mov	r14, rsi
	mov	rbx, rdi
	mov	rdx, qword ptr [rdi + 8]
	add	rdx, -4
	movabs	rax, offset _j_steprange_last_2424
	mov	edi, 1
	mov	esi, 4
	call	rax
	test	rax, rax
	jle	LBB0_1
## %bb.2:                               ## %L19.preheader
	mov	rcx, qword ptr [rbx]
	mov	rdx, qword ptr [r14]
	add	rax, 3
	vxorpd	xmm0, xmm0, xmm0
	xor	esi, esi
	.p2align	4, 0x90
LBB0_3:                                 ## %L19
                                        ## =>This Inner Loop Header: Depth=1
	vmovupd	ymm1, ymmword ptr [rcx + 8*rsi]
	

In [15]:
using BenchmarkTools
@btime vector_dot($B, $C);
@btime vector_dot_unrolled4($B, $C);

  79.043 μs (0 allocations: 0 bytes)
  19.895 μs (0 allocations: 0 bytes)


To "force" automatic SIMD vectorization in Julia, you can use the `@simd` macro.

In [17]:
function vector_dot_simd(B, C)
    a = zero(eltype(B))
    @simd for i in eachindex(B,C)
        @inbounds a += B[i] * C[i]
    end
    return a
end

vector_dot_simd (generic function with 1 method)

By using the `@simd` macro, we are asserting several properties of the loop:

* It is safe to execute iterations in arbitrary or overlapping order, with special consideration for reduction variables.
* Floating-point operations on reduction variables can be reordered, possibly causing different results than without `@simd`.

In [18]:
@btime vector_dot_simd($B, $C);

  18.417 μs (0 allocations: 0 bytes)


This is a huge speedup for just a little extra `@simd`!

In [19]:
@code_native debuginfo=:none syntax=:intel vector_dot_simd(B, C)

	.section	__TEXT,__text,regular,pure_instructions
	.build_version macos, 12, 0
	.globl	_julia_vector_dot_simd_2704     ## -- Begin function julia_vector_dot_simd_2704
	.p2align	4, 0x90
_julia_vector_dot_simd_2704:            ## @julia_vector_dot_simd_2704
	.cfi_startproc
## %bb.0:                               ## %top
	push	rbp
	.cfi_def_cfa_offset 16
	.cfi_offset rbp, -16
	mov	rbp, rsp
	.cfi_def_cfa_register rbp
	push	r15
	push	r14
	push	r13
	push	r12
	push	rbx
	and	rsp, -32
	sub	rsp, 96
	.cfi_offset rbx, -56
	.cfi_offset r12, -48
	.cfi_offset r13, -40
	.cfi_offset r14, -32
	.cfi_offset r15, -24
	mov	r15, rsi
	mov	r12, rdi
	vxorpd	xmm0, xmm0, xmm0
	vmovapd	ymmword ptr [rsp], ymm0
	movabs	rax, 140703379165141
	mov	edi, 260
	vzeroupper
	call	rax
	mov	r14, rax
	mov	qword ptr [rsp], 8
	mov	rax, qword ptr [rax]
	mov	qword ptr [rsp + 8], rax
	mov	rax, rsp
	mov	qword ptr [r14], rax
	mov	rbx, qword ptr [r12 + 8]
	mov	r13, qword ptr [r15 + 8]
	cmp	rbx, r13
	jne	LBB0_10
## %bb.1:               

Note the `vfmadd231pd` instruction and usage of `ymmi` AVX registers (256 bit).

Data types matter:
* Floating-point addition is **non-associative** and the order of operations is important.
* Integer addition is **associative** and the order of operations has no impact.

Let's check what happens for `Int64` input.

In [20]:
B_int = rand(Int64, SIZE)
C_int = rand(Int64, SIZE)
@btime vector_dot($B_int, $C_int);
@btime vector_dot_simd($B_int, $C_int);

  23.150 μs (0 allocations: 0 bytes)
  23.197 μs (0 allocations: 0 bytes)


As expected, there is no difference between the two variants.

### SIMD is hard...

* Autovectorization is a hard problem (it needs to prove a lot of things about the code!)
* Not every code / loop is readily vectorizable
  * Keep your loops simple, e.g. avoid conditionals, control flow, and function calls if possible!
  * Loop length should be countable up front
  * Contiguous data access
  * (Align data structures to SIMD width boundary)

**Keep it simple!**

### [LoopVectorization.jl](https://github.com/JuliaSIMD/LoopVectorization.jl)

Think of `@turbo` as a more sophisticated version of `@simd`. Hopefully, these features will at some point just be part of Julia's compiler.

In [23]:
using LoopVectorization

function vector_dot_turbo(B, C)
    a = zero(eltype(B))
    @turbo for i in eachindex(B,C)
        @inbounds a += B[i] * C[i]
    end
    return a
end

@btime vector_dot_simd($B, $C);
@btime vector_dot_turbo($B, $C);

  18.439 μs (0 allocations: 0 bytes)
  18.416 μs (0 allocations: 0 bytes)


In [24]:
@code_native debuginfo=:none syntax=:intel vector_dot_turbo(B, C)

	.section	__TEXT,__text,regular,pure_instructions
	.build_version macos, 12, 0
	.section	__TEXT,__const
	.p2align	5                               ## -- Begin function julia_vector_dot_turbo_6143
LCPI0_0:
	.quad	0                               ## 0x0
	.quad	1                               ## 0x1
	.quad	2                               ## 0x2
	.quad	3                               ## 0x3
	.section	__TEXT,__text,regular,pure_instructions
	.globl	_julia_vector_dot_turbo_6143
	.p2align	4, 0x90
_julia_vector_dot_turbo_6143:           ## @julia_vector_dot_turbo_6143
	.cfi_startproc
## %bb.0:                               ## %top
	push	rbp
	.cfi_def_cfa_offset 16
	.cfi_offset rbp, -16
	mov	rbp, rsp
	.cfi_def_cfa_register rbp
	push	r15
	push	r14
	push	r13
	push	r12
	push	rbx
	and	rsp, -32
	sub	rsp, 96
	.cfi_offset rbx, -56
	.cfi_offset r12, -48
	.cfi_offset r13, -40
	.cfi_offset r14, -32
	.cfi_offset r15, -24
	mov	r15, rsi
	mov	rbx, rdi
	vxorpd	xmm0, xmm0, xmm0
	vmovapd	ymmword ptr [rsp], ymm0
	

Note the usage of the `zmmi` AVX512 registers! (512 bit)

## Structure of Array vs Array of Structure

Data layout can matter!

In [ ]:
# Array of structure
AoS = [complex(rand(),rand()) for i in 1:SIZE]

In [ ]:
@btime sum($AoS);

### [StructArrays.jl](https://github.com/JuliaArrays/StructArrays.jl)

In [ ]:
using StructArrays

In [ ]:
SoA = StructArray{Complex}((rand(SIZE), rand(SIZE)))

In [ ]:
@btime sum($SoA);

**Resources:**

* [LoopVectorization.jl video on youtube](https://www.youtube.com/watch?v=qz2kJdVDWi0)
* [SIMD and SIMD-intrinsics in Julia](http://kristofferc.github.io/post/intrinsics/)
* [Optimizing Serial Code](https://mitmath.github.io/18337/lecture2/optimizing)